# Automação de Sistemas e Processos com Python

### Desafio:

Diariamente, o nosso sistema atualiza as vendas do dia anterior.
O seu trabalho diário, como analista, é enviar um e-mail para a diretoria, assim que começar a trabalhar, com o faturamento e a quantidade de produtos vendidos no dia anterior

E-mail da diretoria: seugmail+diretoria@gmail.com<br>
Local onde o sistema disponibiliza as vendas do dia anterior: https://drive.google.com/drive/folders/149xknr9JvrlEnhNWO49zPcw0PW5icxga?usp=sharing

Para resolver isso, vamos usar o pyautogui, uma biblioteca de automação de comandos do mouse e do teclado

In [2]:
import pyautogui
import pyperclip
from time import sleep
from datetime import datetime
import pandas as pd
from os import remove

In [12]:
pyautogui.PAUSE = 2
FECHAR = True

In [13]:
# Passo 1: Entrar no sistema da empresa (no nosso caso é o link do drive)

# abrir navegador
sleep(0.5)
pyautogui.press('win')
pyautogui.write('opera')
pyautogui.press('enter')
while not pyautogui.locateOnScreen('images/navegador.png', confidence=0.8):
    pyautogui.hotkey('ctrl', 'w')
    sleep(1)

# entrar no site
pyperclip.copy(
    'https://drive.google.com/drive/folders/149xknr9JvrlEnhNWO49zPcw0PW5icxga?usp=sharing')
pyautogui.hotkey('ctrl', 'v')
pyautogui.press('enter')
while not pyautogui.locateOnScreen('images/drive.png', confidence=0.9):
    sleep(1)

# Passo 2: Navegar no sistema e encontrar a base de vendas (entrar na pasta exportar)
pyautogui.click(x=360, y=255, clicks=2)
while not pyautogui.locateOnScreen('images/vendas.png', confidence=0.9):
    sleep(1)

# Passo 3: Fazer o download da base de vendas
pyautogui.click(x=364, y=258)
pyautogui.click(x=1156, y=160)
while not pyautogui.locateOnScreen('images/fazer_download.png', confidence=0.9):
    sleep(1)
pyautogui.click(x=958, y=562)
while not pyautogui.locateOnScreen('images/download.png', confidence=0.9):
    sleep(1)

### Vamos agora ler o arquivo baixado para pegar os indicadores

- Faturamento
- Quantidade de Produtos

In [3]:
# Passo 4: Importar a base de vendas para o Python
df = pd.read_excel(r'C:\Users\f4dra\Downloads\Vendas - Dez.xlsx')
display(df)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,65014,2019-12-01,Shopping Morumbi,Sunga Listrado,5,114,570
1,65014,2019-12-01,Shopping Morumbi,Casaco Listrado,1,269,269
2,65016,2019-12-01,Iguatemi Campinas,Sapato Listrado,2,363,726
3,65016,2019-12-01,Iguatemi Campinas,Casaco,1,250,250
4,65017,2019-12-01,Shopping SP Market,Gorro Liso,3,92,276
...,...,...,...,...,...,...,...
7084,69996,2019-12-26,Center Shopping Uberlândia,Short Listrado,2,102,204
7085,69996,2019-12-26,Center Shopping Uberlândia,Mochila,4,270,1080
7086,69996,2019-12-26,Center Shopping Uberlândia,Pulseira Estampa,1,87,87
7087,69997,2019-12-26,Ribeirão Shopping,Camisa Listrado,1,108,108


In [15]:
# Passo 5: Calcular os indicadores
faturamento = df['Valor Final'].sum()
media_lucro = df['Valor Final'].mean()
qtd_produtos = df['Quantidade'].sum()
list_lojas = list(set(df['ID Loja']))
list_produtos = list(set(df['Produto']))
melhor = df.max()
pior = df.min()
produtos = [(df['Produto'][i], list(df['Produto']).count(list_produtos[i]))
            for i in range(len(list_produtos))]
lojas = [(df['ID Loja'][i], list(df['ID Loja']).count(list_lojas[i]))
         for i in range(len(list_lojas))]
mais_vendido = max(produtos, key=lambda p: p[1])
menos_vendido = min(produtos, key=lambda p: [1])
melhor_loja = max(lojas, key=lambda loja: loja[1])
pior_loja = min(lojas, key=lambda loja: loja[1])
hora_atual = datetime.now().hour

In [16]:
if hora_atual < 12:
    cumprimento = 'Bom dia'
elif hora_atual < 18:
    cumprimento = 'Boa tarde'
else:
    cumprimento = 'Boa noite'

### Vamos agora enviar um e-mail pelo gmail

In [17]:
# Passo 5: Entrar no email
pyautogui.hotkey('ctrl', 'w')
pyperclip.copy('https://mail.google.com/mail/u/0/')
pyautogui.hotkey('ctrl', 'v')
pyautogui.press('enter')
while not pyautogui.locateOnScreen('images/escrever.png', confidence=0.9):
    sleep(1)

# Passo 6: Enviar pro email o resultado
pyautogui.click(x=117, y=171)
while not pyautogui.locateOnScreen('images/mensagem.png', confidence=0.9):
    sleep(1)

pyautogui.write('pythonimpressionador+diretoria@gmail.com')
pyautogui.press('tab')  # seleciona o email

pyautogui.press('tab')  # pula para o campo de assunto
pyperclip.copy('Relatório de Vendas')
pyautogui.hotkey('ctrl', 'v')  # escreve o assunto
pyautogui.press('tab')  # pula para o corpo do email

texto = f'''Prezados, {cumprimento}

Segue relatório de vendas.
Faturamento total: R${faturamento:,.2f}
Média de faturamento por venda: R${media_lucro:,.2f}
Quantidade de produtos vendidos: {qtd_produtos:,}
Quantidade de lojas no momento: {len(list_lojas)}
Melhor venda:
    Data: {melhor['Data'].day}/{melhor['Data'].month}/{melhor['Data'].year}
    ID Loja: {melhor['ID Loja']}
    Produto: {melhor['Produto']}
    Quantidade: {melhor['Quantidade']:,}
    Valor Unitário: {melhor['Valor Unitário']:,.2f}
    Valor Final: R${melhor['Valor Final']:,.2f}
Pior venda:
    Data: {melhor['Data'].day}/{melhor['Data'].month}/{melhor['Data'].year}
    ID Loja: {pior['ID Loja']}
    Produto: {pior['Produto']}
    Quantidade: {pior['Quantidade']:,}
    Valor Unitário: {pior['Valor Unitário']:,.2f}
    Valor Final: R${pior['Valor Final']:,.2f}
Mais vendido:
    Nome: {mais_vendido[0]}
    Quantidade: {mais_vendido[1]}
Menos vendido:
    Nome: {menos_vendido[0]}
    Quantidade: {menos_vendido[1]}
Melhor loja:
    ID Loja: {melhor_loja[0]}
    Produtos vendidos: {melhor_loja[1]}
Pior loja:
    ID Loja: {pior_loja[0]}
    Produtos vendidos: {pior_loja[1]}

Qualquer dúvida estou à disposição.
Att.,
Felippe do Python'''

pyperclip.copy(texto)
pyautogui.hotkey('ctrl', 'v')

# anexar um arquivo
arquivo = r'C:\Users\f4dra\Downloads\Vendas - Dez.xlsx'
pyautogui.click(x=952, y=702)
pyautogui.press('enter')
while not pyautogui.locateOnScreen('images/nome.png', confidence=0.9):
    sleep(1)
pyperclip.copy(arquivo)
pyautogui.hotkey('ctrl', 'v')
pyautogui.press('enter')
pyautogui.click(x=1093, y=464)
while not pyautogui.locateOnScreen('images/arquivo_completo.png', confidence=0.9):
    sleep(1)

# enviar email
pyautogui.hotkey('ctrl', 'enter')

In [18]:
try:
    remove(r'C:\Users\f4dra\Downloads\Vendas - Dez.xlsx')
except FileNotFoundError:
    print('\033[31mArquivo não encontrado.\033[m')

In [19]:
if FECHAR:
    while not pyautogui.locateOnScreen('images/mensagem_enviada.png', confidence=0.9):
        sleep(1)
    pyautogui.hotkey('ctrl', 'w')
    pyautogui.hotkey('alt', 'f4')

In [21]:
print('\033[32mPROGRAMA FINALIZADO COM SUCESSO!\033[m')

PROGRAMA FINALIZADO COM SUCESSO!


#### E se eu quisesse anexar arquivos?

arquivo = r'C:\Users\f4dra\Downloads\Vendas - Dez.xlsx'
pyautogui.click(x=952, y=702)
pyautogui.press('enter')
sleep(2)
pyperclip.copy(arquivo)
pyautogui.hotkey('ctrl', 'v')
pyautogui.press('enter')
sleep(3)

#### E se ao invés de esperar alguns segundos eu quisesse reconhecer uma imagem na tela?

sleep(3)

#### entrar no site
pyperclip.copy('https://drive.google.com/drive/folders/149xknr9JvrlEnhNWO49zPcw0PW5icxga?usp=sharing')
pyautogui.hotkey('ctrl', 'v')
pyautogui.press('enter')

while not pyautogui.locateOnScreen('images/drive.png', confidence=0.9): sleep(1)

#### clicar na pasta
pyautogui.click(x=360, y=255, clicks=2)

#### Use esse código para descobrir qual a posição de um item que queira clicar

- Lembre-se: a posição na sua tela difere da posição na minha tela